# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [4]:
%%writefile ../flows/cloud/event_triggered_linear_regression.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, retry, timeout, project
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

@project(name="taxi_fare_prediction")
@trigger(events=["s3"])
@conda_base(
    libraries={
        "pandas": "1.4.2",
        "pyarrow": "11.0.0",
        #"numpy": "1.21.2",
        "numpy": "1.22.0",
        "scikit-learn": "1.1.2",
    }
)
class TaxiFarePrediction(FlowSpec):
    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # TODO:
        # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
        # Understand what is happening.
        # Revisit task 1 and think about what might go in this function.
        obviously_bad_data_filters = [
            df.fare_amount > 0,  # fare_amount in US Dollars
            df.trip_distance <= 100,  # trip_distance in miles
            df.trip_distance > 0,
            df.tip_amount >= 0,
            df.total_amount >= 0,
            df.passenger_count >= 0
            ]

        print('Starting Data Size: {0}'.format(len(df)))
        print('Removing Bad Data')
        for f in obviously_bad_data_filters:
            df = df[f]
        print('Ending Data Size: {0}'.format(len(df)))
        #df = df[~(df['trip_distance'].isnull())] # Remove missing values from dataset

        return df

    @step
    def start(self):
        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests.
        self.X = self.df[["trip_distance", "passenger_count"]]
        self.y = self.df["total_amount"].values
        print("Missing Value count is {} for trip distance".format(sum(self.df["trip_distance"].isnull())))
        print("Missing Value count is {} for y".format(sum(self.df["total_amount"].isnull())))

        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        # TODO: Play around with the model if you are feeling it.
        self.model = LinearRegression()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):
        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅"
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [
                    Markdown(icon),
                    Artifact(run.id),
                    Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                    Artifact(score),
                    Markdown(msg),
                ]
                rows.append(row)
            else:
                rows.append(
                    [
                        Markdown("✅"),
                        Artifact(run.id),
                        Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                        Artifact(str(self.scores.mean())),
                        Markdown("This run..."),
                    ]
                )
        return rows

    @card(type="corise")
    @retry
    @timeout(minutes=10)
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score

        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(
            Table(
                self.gather_sibling_flow_run_results(),
                headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"],
            )
        )
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/event_triggered_linear_regression.py


In [2]:
# Test Flow Runs
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda run

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2023-11-10 18:29:41.344 Creating local datastore in current directory (/home/workspace/workspaces/full-stack-ml-metaflow-corise-week-4/project/.metaflow)
Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
2023-11-10 18:32:22.568 Workflow starting (run-id 428), see it in the UI at https://ui-pw-2038312917.outerbounds.dev/TaxiFarePrediction/428
2023-11-10 18:32:23.230 [428/start/2128 (pid 14414)] Task is starting.
2023-11-10 18:32:30.570 [428/start/2128 (pid 14414)] Starting Data Size: 962487
2023-11-10 18:32:33.741 [428/start/2128 (pid 14414)] Removing Bad Data
2023-11-10 18:32:33.742 [428/start/2128 (pid 14414)] Ending Data Size: 916154
2023-11-10 18:32:33.943 [428/start/2128 (pid 14414)] Missing Value count is 0 for trip distance
2023-11-10 18:32:34.081 [428/start/2128 (pid 14414)] Missing Value count i

In [5]:
# Deploy Champion Model to Argo
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production \
    --branch champion --with retry argo-workflows create 

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.champion.taxifareprediction to Argo Workflows...
It seems this is the first time you are deploying taxifareprediction.prod.champion.taxifareprediction to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-ovzw7jjg7psagpyw-0-pcke
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-ovzw7jjg7psagpyw-0-pcke")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-ovzw7jjg7psagpyw-0-pcke
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about produ

In [10]:
# Manually Trigger Workflow
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda \
    --production --branch champion argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.champion.taxifareprediction-zlch7).
See the run in the UI at https://ui-pw-2038312917.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.champion.taxifareprediction-zlch7


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [16]:
%%writefile ../flows/cloud/event_triggered_linear_regression_challenger.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, retry, timeout, project
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

@project(name="taxi_fare_prediction")
@trigger(events=["s3"])
@conda_base(
    libraries={
        "pandas": "1.4.2",
        "pyarrow": "11.0.0",
        #"numpy": "1.21.2",
        "numpy": "1.22.0",
        "scikit-learn": "1.1.2",
    }
)
class TaxiFarePrediction(FlowSpec):
    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # TODO:
        # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
        # Understand what is happening.
        # Revisit task 1 and think about what might go in this function.
        obviously_bad_data_filters = [
            df.fare_amount > 0,  # fare_amount in US Dollars
            df.trip_distance <= 100,  # trip_distance in miles
            df.trip_distance > 0,
            df.tip_amount >= 0,
            df.total_amount >= 0,
            df.passenger_count >= 0
            ]

        print('Starting Data Size: {0}'.format(len(df)))
        print('Removing Bad Data')
        for f in obviously_bad_data_filters:
            df = df[f]
        print('Ending Data Size: {0}'.format(len(df)))
        # Add Transformed Features
        df['trip_sq'] = df['trip_distance']*df['trip_distance']
        df['pass_sq'] = df['passenger_count']*df['passenger_count']
        df['trip_pass_int'] = df['trip_distance']*df['passenger_count']

        return df

    @step
    def start(self):
        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests.
        self.X = self.df[["trip_distance", "passenger_count", 
                          "trip_sq", "pass_sq", "trip_pass_int"]]
        self.y = self.df["total_amount"].values
        print("Missing Value count is {} for trip distance".format(sum(self.df["trip_distance"].isnull())))
        print("Missing Value count is {} for y".format(sum(self.df["total_amount"].isnull())))

        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import Ridge

        # TODO: Play around with the model if you are feeling it.
        self.model = Ridge(alpha=.5)

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):
        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅"
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [
                    Markdown(icon),
                    Artifact(run.id),
                    Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                    Artifact(score),
                    Markdown(msg),
                ]
                rows.append(row)
            else:
                rows.append(
                    [
                        Markdown("✅"),
                        Artifact(run.id),
                        Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                        Artifact(str(self.scores.mean())),
                        Markdown("This run..."),
                    ]
                )
        return rows

    @card(type="corise")
    @retry
    @timeout(minutes=10)
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score

        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(
            Table(
                self.gather_sibling_flow_run_results(),
                headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"],
            )
        )
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/event_triggered_linear_regression_challenger.py


In [17]:
# Test Flow Runs
! python ../flows/cloud/event_triggered_linear_regression_challenger.py --environment=conda run

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
2023-11-10 19:07:09.966 Workflow starting (run-id 432), see it in the UI at https://ui-pw-2038312917.outerbounds.dev/TaxiFarePrediction/432
2023-11-10 19:07:10.534 [432/start/2148 (pid 15640)] Task is starting.
2023-11-10 19:07:18.039 [432/start/2148 (pid 15640)] Starting Data Size: 969925
2023-11-10 19:07:21.063 [432/start/2148 (pid 15640)] Removing Bad Data
2023-11-10 19:07:21.064 [432/start/2148 (pid 15640)] Ending Data Size: 923281
2023-11-10 19:07:21.850 [432/start/2148 (pid 15640)] Missing Value count is 0 for trip distance
2023-11-10 19:07:22.051 [432/start/2148 (pid 15640)] Missing Value count is 0 for y
2023-11-10 19:07:39.524 [432/start/2148 (pid 15640)] Task finished successfully.
2023-11-10 

In [18]:
# Deploy Challenger Model to Argo
! python ../flows/cloud/event_triggered_linear_regression_challenger.py --environment=conda \
    --production --branch challenger --with retry argo-workflows create 

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.challenger.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-6iffsxtybx6fkjku-0-crtx
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-6iffsxtybx6fkjku-0-crtx")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-6iffsxtybx6fkjku-0-crtx
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
Workflow taxifareprediction.prod.challenger.taxifarepredict

In [19]:
# Manually Trigger Workflow
! python ../flows/cloud/event_triggered_linear_regression_challenger.py --environment=conda \
    --production --branch challenger argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.challenger.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.challenger.taxifareprediction-xm97x).
See the run in the UI at https://ui-pw-2038312917.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-xm97x


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [23]:
from metaflow import Flow, namespace

# these values are unique to your deployment!
CHAMPION_MODEL_NAMESPACE = 'production:mfprj-ovzw7jjg7psagpyw-0-pcke'
CHALLENGER_MODEL_NAMESPACE = 'production:mfprj-6iffsxtybx6fkjku-0-crtx' 

for space in [CHAMPION_MODEL_NAMESPACE, CHALLENGER_MODEL_NAMESPACE]:
    namespace(space)
    run = Flow('TaxiFarePrediction').latest_successful_run
    scores = run.data.scores.mean()
    if space == CHAMPION_MODEL_NAMESPACE:
        champ_scores = scores
    else:
        chall_scores = scores
    print('{0}:{1}'.format(space, scores))

if champ_scores > chall_scores:
    print('Champion model wins with score of {}'.format(champ_scores))
else:
    print('Challnger model wins with score of {}'.format(chall_scores))


production:mfprj-ovzw7jjg7psagpyw-0-pcke:0.8963641800487778
production:mfprj-6iffsxtybx6fkjku-0-crtx:0.8964484611416204
Challnger model wins with AUC of 0.8964484611416204


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!